In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor, PILToTensor
from torchvision.transforms.functional import to_pil_image
import torch.nn.functional as F


In [4]:
# Workaround to download mnist without 403 error https://github.com/pytorch/vision/issues/1938#issuecomment-594623431
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [50]:
def show_tensor_as_image(tensor):
    image = to_pil_image(tensor[0])
    display(image)

In [6]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
batch_size = 64
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [23]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvNeuralNetwork(nn.Module):

    def __init__(self):
        super(ConvNeuralNetwork, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(400, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output

class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )

    def forward(self, input):
        x1 = torch.flatten(input, 1)
        x2 = self.linear_relu_stack(x1)
        return x2
model = ConvNeuralNetwork().to(device)
print(model)

Using cuda device
ConvNeuralNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309274  [   64/60000]
loss: 2.298903  [ 6464/60000]
loss: 2.297908  [12864/60000]
loss: 2.293134  [19264/60000]
loss: 2.290456  [25664/60000]
loss: 2.289624  [32064/60000]
loss: 2.307314  [38464/60000]
loss: 2.312172  [44864/60000]
loss: 2.286929  [51264/60000]
loss: 2.290781  [57664/60000]
Test Error: 
 Accuracy: 13.0%, Avg loss: 2.296296 

Epoch 2
-------------------------------
loss: 2.300026  [   64/60000]
loss: 2.289924  [ 6464/60000]
loss: 2.290345  [12864/60000]
loss: 2.284423  [19264/60000]
loss: 2.286300  [25664/60000]
loss: 2.284458  [32064/60000]
loss: 2.295410  [38464/60000]
loss: 2.302509  [44864/60000]
loss: 2.280283  [51264/60000]
loss: 2.280836  [57664/60000]
Test Error: 
 Accuracy: 16.2%, Avg loss: 2.287426 

Epoch 3
-------------------------------
loss: 2.290981  [   64/60000]
loss: 2.281035  [ 6464/60000]
loss: 2.281439  [12864/60000]
loss: 2.275071  [19264/60000]
loss: 2.279869  [25664/60000]
loss: 2.276867  [32064/600

In [28]:
torch.save(model.state_dict(), "model_weights.pth")

In [30]:
model = ConvNeuralNetwork().to(device)
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()
test(test_dataloader, model, loss_fn)


/tmp/ipykernel_121443/3791756519.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_weights.pth"))


Test Error: 
 Accuracy: 96.7%, Avg loss: 0.109978 



In [51]:
for data,labels in test_dataloader:
    X, y = data.to(device), labels.to(device)
    print(X.shape)
    pred = model(X)
    for image,label,prediction in list(zip(data,labels,pred))[:10]:
        image = to_pil_image(image)
        print("True label:",label.item(),"prediction:",torch.argmax(prediction).item())
        display(image)
    break

torch.Size([64, 1, 28, 28])
True label: 7 prediction: 7


True label: 2 prediction: 2


True label: 1 prediction: 1


True label: 0 prediction: 0


True label: 4 prediction: 4


True label: 1 prediction: 1


True label: 4 prediction: 4


True label: 9 prediction: 9


True label: 5 prediction: 5


True label: 9 prediction: 9


In [52]:
display(image)
transform = PILToTensor()

original_image = transform(image)/255
original_image = original_image.clone()

In [108]:
def generate_adversarial_image(input_image,target):
    transform = PILToTensor()
    original_image = transform(input_image)/255
    original_image = original_image.clone()
    X = original_image.unsqueeze(0).to(device)
    y = torch.tensor(target).unsqueeze(0).to(device)
    X, y = X.to(device), y.to(device)
    perturbation = torch.zeros(X.shape,requires_grad=True,device=device)
    optimizer = torch.optim.Adam([perturbation], lr=1)
    
    loss_fn = nn.CrossEntropyLoss()
    epsilon = 0.1
    model.eval()
    for i in range(20):
        # Compute prediction error
        adversarial_input = X+perturbation
        adversarial_input = torch.clamp(adversarial_input, 0, 1)

        pred = model(adversarial_input)
        loss = loss_fn(pred, y)
        loss.backward()
        #print(f"Iteration {i}, Perturbation Norm: {perturbation.norm().item()}")
    
        optimizer.step()
        with torch.no_grad():
            perturbation.clamp_(-epsilon, epsilon)
        optimizer.zero_grad()
    image_tensor = (X+perturbation)[0].clamp_(0,1)
    return image_tensor,perturbation

In [109]:
adversarial_image,perturbation = generate_adversarial_image(image,8)
print(adversarial_image.shape)
pred = model(adversarial_image.unsqueeze(0))
print(torch.argmax(pred).item(), pred[0].tolist())
show_tensor_as_image(adversarial_image)

torch.Size([1, 28, 28])
8 [0.8722741603851318, -7.8081464767456055, -0.41491296887397766, -0.6415470838546753, 1.2746644020080566, -0.9251648783683777, -2.071305990219116, 2.277597427368164, 5.24390983581543, 4.65758752822876]


In [87]:
def fastGradientSignMethod(input_image, target_label, model,epsilon):
    loss_fn = nn.CrossEntropyLoss()
    model.eval()
    transform = PILToTensor()
    original_image = transform(input_image)/255
    original_image = original_image.clone()
    X = original_image.unsqueeze(0).to(device)
    #print(X)
    y = torch.tensor(target_label).unsqueeze(0).to(device)

    perturbation = torch.zeros(X.shape,requires_grad=True,device=device)
    adversarial_input = X+perturbation
    pred = model(adversarial_input)
    print(pred[0][target_label].item())
    loss = loss_fn(pred, y)
    loss.backward()
    with torch.no_grad():
        perturbation = epsilon * perturbation.grad.sign()
        adversarial_input = X + perturbation
    #print(perturbation)
    return adversarial_input.clamp_(0,1)

In [88]:
adversarial_image = fastGradientSignMethod(image,8,model,0.1)
pred = model(adversarial_image)
print(torch.argmax(pred).item(), pred[0].tolist())
show_tensor_as_image(adversarial_image)

3.4972147941589355
9 [1.5764107704162598, -10.002506256103516, 0.23172877728939056, 0.6589263081550598, 2.89674973487854, -1.873987078666687, -8.626487731933594, 7.821351051330566, 2.171248435974121, 10.553485870361328]


In [37]:
display(image)